In [ ]:
import pandas as pd
from neo4j import GraphDatabase
# 读取数据，这里的数据通过dataprocess.ipynb处理后得到
# Generator_TX : 德州所有发电厂数据
# Solar_TX : 德州太阳能发电厂数据
# Wind_TX : 德州风力发电厂数据
# Other_generator_TX : 德州其他类型发电厂数据
# Multifuel_TX : 德州多燃料发电厂数据
# Energy_Storage_TX : 德州所有储能厂数据
Generator_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Generator_TX.csv')
Solar_TX = pd.read_csv('/home/lijh/knowledge-graph/TX_data/Solar_TX.csv')
# 这个类是用来连接到Neo4j数据库的，从官方文档中复制过来的
class KnowledgeGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_relationship(self, node1, node2, relationship_type):
        with self.driver.session() as session:
            session.execute_write(self._create_and_return_relationship, node1, node2, relationship_type)

    @staticmethod
    def _create_and_return_relationship(tx, node1, node2, relationship_type):
        query = (
            "MATCH (a {name: $node1_name}), (b:County {name: $node2_name}) "
            f"MERGE (a)-[r:{relationship_type}]->(b) "
            "RETURN type(r)"
        )
        tx.run(query, node1_name=node1, node2_name=node2)
# 以下两个函数是针对此项目自建的函数，用于遍历DataFrame中的每一行，循环调用官方create函数创建节点和关系
# 此函数是创建各种机组节点，这里的节点是发电厂机组，包含了机组名称和技术类型
def create_nodes(session, df, label):
    for index, row in df.iterrows():
        name = f"{row['Plant Name']}_{row['Generator ID']}"
        session.run(
            f"MERGE (n:{label} {{name: $name, technology: $technology}})", 
            name=name, technology=row['Technology']
        )
# 此函数是建立各种机组和县的所在关系，这里的关系是located_in，表示机组位于某个县
def create_relationships(session, df, label, kg):
    for index, row in df.iterrows():
        node_name = f"{row['Plant Name']}_{row['Generator ID']}"
        county_name = row['County']
        Utility_name = row['Utility Name']
        kg.create_relationship(node_name, county_name, "located_in")
        kg.create_relationship(node_name, Utility_name, "owned_by")
# 主函数，创建KnowledgeGraph对象，连接到数据库，调用上面的函数，最后关闭连接
if __name__ == "__main__":
    kg = KnowledgeGraph("neo4j+s://0887a3e7.databases.neo4j.io", "neo4j", "fZ9mI0LlBpKhp2HE2Oa_ZLy3j0gQQmtgAnWGl8bBNtE")
    # 创建一个session，用于执行cypher语句，with语句保证了session的自动关闭
    with kg.driver.session() as session:
        existing_counties = set(record['name'] for record in session.run("MATCH (c:County) RETURN c.name AS name"))
        existing_utilities = set(record['name'] for record in session.run("MATCH (c:Utility) RETURN c.name AS name"))
        # 添加County节点，此处设置了一个外部的列表，不断创建过程中不断更新，防止重复创建county节点
        for county_name in Generator_TX['County'].unique():
            if county_name not in existing_counties:
                session.run("MERGE (c:County {name: $name})", name=county_name)
                existing_counties.add(county_name)
        # 添加Utility节点，此处设置了一个外部的列表，不断创建过程中不断更新，防止重复创建Utility节点
        for Utility_name in Generator_TX['Utility Name'].unique():
            if Utility_name not in existing_utilities:
                session.run("MERGE (c:Utility {name: $name})", name=Utility_name)
                existing_utilities.add(Utility_name)
        # 调用创建节点的函数
        create_nodes(session, Solar_TX, 'Solar')
        # 调用创建关系的函数
        create_relationships(session, Solar_TX, 'Solar', kg)
    # 关闭连接
    kg.close()